In [ ]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
# Load training data
with open("generator_training_data.pkl", "rb") as f:
    data = pickle.load(f)

network_input = data["input"]
network_output = data["output"]   # not used here, for generator
network_mood = data["mood"]

note_to_int = data["note_to_int"]
int_to_note = data["int_to_note"]
mood_to_int = data["mood_to_int"]
int_to_mood = data["int_to_mood"]
SEQ_LEN = data["seq_len"]

n_vocab = len(note_to_int)
n_moods = len(mood_to_int)

print(f"Training on {network_input.shape[0]} sequences")
print(f"Vocabulary size: {n_vocab}, Mood classes: {n_moods}")

In [ ]:
# Build LSTM classifier model
model = Sequential()
model.add(Embedding(input_dim=n_vocab, output_dim=100, input_length=SEQ_LEN))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(128, activation="relu"))
model.add(Dense(n_moods, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

model.summary()

In [ ]:
# Save the best model
checkpoint = ModelCheckpoint(
    "D:\Study\Ai_Music_Composer\project\models\lstm_mood_classifier.h5",
    monitor="val_accuracy",
    save_best_only=True,
    mode="max",
    verbose=1
)

In [ ]:
# Train the classifier
history = model.fit(
    network_input, network_mood,
    validation_split=0.2,
    epochs=30,
    batch_size=64,
    callbacks=[checkpoint]
)

print("✅ Training finished. Best model saved as lstm_mood_classifier.h5")